<a href="https://colab.research.google.com/github/petroDavydov/goit-DeepLearningForComputerVisionAndNLP/blob/main/HW_PyTorchBasics_4_PetroDavydov.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from dataclasses import dataclass

import os
import warnings

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, TensorDataset


# ***2. Підготовка даних***

In [ ]:
df = pd.read_csv('/content/ConcreteStrengthData.csv')
df

In [ ]:
df.info()

# ***2. Підготовкка даних***

In [ ]:
X = df.drop(columns=['Strength'])   # ознаки
y = df['Strength'].values           # цільова змінна

In [ ]:
# train/test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# нормалізація
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [ ]:
# перетворення у тензори
X_train_t = torch.tensor(X_train, dtype=torch.float32)
y_train_t = torch.tensor(y_train, dtype=torch.float32).unsqueeze(1)
X_test_t = torch.tensor(X_test, dtype=torch.float32)
y_test_t = torch.tensor(y_test, dtype=torch.float32).unsqueeze(1)

In [ ]:
# Dataset і DataLoader
train_dataset = TensorDataset(X_train_t, y_train_t)
test_dataset = TensorDataset(X_test_t, y_test_t)
batch_size = 64
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [ ]:
# 3. Створення моделі
class ConcreteNet(nn.Module):
    def __init__(self, in_dim):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(in_dim, 128),
            nn.ReLU(),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Linear(64, 1)   # вихідний шар для регресії
        )
    def forward(self, x):
        return self.net(x)

In [ ]:
# 4. Налаштування навчання
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = ConcreteNet(in_dim=X_train.shape[1]).to(device)
criterion = nn.MSELoss()   # функція втрат для регресії
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
num_epochs = 100

In [ ]:
# 5. Навчання моделі
train_losses = []
for epoch in range(1, num_epochs+1):
    model.train()
    epoch_losses = []
    for X_batch, y_batch in train_loader:
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)
        optimizer.zero_grad()
        outputs = model(X_batch)
        loss = criterion(outputs, y_batch)
        loss.backward()
        optimizer.step()
        epoch_losses.append(loss.item())
    train_losses.append(np.mean(epoch_losses))

    if epoch % 10 == 0:
        model.eval()
        with torch.no_grad():
            preds, trues = [], []
            for Xb, yb in test_loader:
                Xb = Xb.to(device)
                out = model(Xb)
                preds.extend(out.cpu().numpy().flatten())
                trues.extend(yb.numpy().flatten())
            preds, trues = np.array(preds), np.array(trues)
            rmse = np.sqrt(mean_squared_error(trues, preds))
        print(f'Epoch [{epoch}/{num_epochs}], Loss: {train_losses[-1]:.4f}, RMSE: {rmse:.6f}')

In [ ]:
# 6. Оцінка моделі
model.eval()
with torch.no_grad():
    y_pred = model(X_test_t.to(device)).cpu().numpy().flatten()
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
print('MSE:', mse)
print('MAE:', mae)
print('R2:', r2)

In [ ]:
# 7. Аналіз результатів
print()
plt.figure(figsize=(6,6))
plt.scatter(y_test, y_pred, alpha=0.6)
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--')
plt.xlabel('Actual Strength')
plt.ylabel('Predicted Strength')
plt.title('Actual vs Predicted')
plt.show()
print()
plt.figure()
plt.plot(train_losses)
plt.xlabel('Epoch')
plt.ylabel('Train Loss (MSE)')
plt.title('Training Loss Curve')
plt.show()

# ***Висновки по першій спробі до оптимізації***

Отримані результати
MSE ≈ 257.7 - значно вище за «непогані» (75–55).

MAE ≈ 13.0 - теж вище за «непогані» (10–8).

R² ≈ 0.0 - модель практично не пояснює варіацію даних.

# ***Аналіз***

1. Чому модель не досягла очікуваних показників

 - Архітектура занадто проста для складної нелінійної залежності між компонентами бетону та міцністю.

 - Можливо, дані мають сильний шум або нелінійні взаємозв’язки, які лінійні шари з ReLU не вловили.

 - Гіперпараметри (SGD з lr=0.01, 100 епох) могли бути недостатніми для якісної збіжності.


2. Критична оцінка архітектури

 - Використано 2 приховані шари (128 → 64). Це базова структура, яка може бути замалою.

 - Відсутні регуляризація (Dropout, L2), що могло призвести до нестабільності.

 - Оптимізатор SGD сходиться повільніше, ніж Adam чи RMSprop.

3. Шляхи покращення

 - Спробувати глибшу архітектуру (наприклад, 256 → 128 → 64 → 32).

 - Використати Adam як оптимізатор, він краще працює для регресії з шумними даними.

 - Збільшити кількість епох (200–300) або зменшити learning rate.

 - Додати регуляризацію (Dropout, L2 weight decay).

 - Перевірити кореляції між ознаками, можливо, деякі компоненти неінформативні.

------------------------------------------------------------------------


Модель навчилась, але її результати не відповідають очікуваним метрикам. Це свідчить про те, що базова архітектура та вибір оптимізатора були недостатніми для задачі. Проте код виконується, дані підготовлені, цикл навчання реалізований, метрики обчислені. Для покращення варто експериментувати з архітектурою, оптимізатором та гіперпараметрами.

-----------------------------------------------------------------------

# ***Варіант максимальної оптимізації, для експерименту***

In [ ]:
# 8. Оптимізація моделі (приклад альтернативної архітектури)
class ConcreteNetDeep(nn.Module):
    def __init__(self, in_dim):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(in_dim, 512),
            nn.ReLU(),
            nn.Linear(512, 256),
            nn.ReLU(),
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.Linear(32, 16),
            nn.ReLU(),
            nn.Linear(16, 8),
            nn.ReLU(),
            nn.Linear(8, 4),
            nn.ReLU(),
            nn.Linear(4, 1)
        )
    def forward(self, x):
        return self.net(x)

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = ConcreteNet(in_dim=X_train.shape[1]).to(device)
criterion = nn.MSELoss()   # функція втрат для регресії
optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-4)
num_epochs = 400

In [ ]:
# Навчання моделі
train_losses = []
for epoch in range(1, num_epochs+1):
    model.train()
    epoch_losses = []
    for X_batch, y_batch in train_loader:
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)
        optimizer.zero_grad()
        outputs = model(X_batch)
        loss = criterion(outputs, y_batch)
        loss.backward()
        optimizer.step()
        epoch_losses.append(loss.item())
    train_losses.append(np.mean(epoch_losses))

    if epoch % 10 == 0:
        model.eval()
        with torch.no_grad():
            preds, trues = [], []
            for Xb, yb in test_loader:
                Xb = Xb.to(device)
                out = model(Xb)
                preds.extend(out.cpu().numpy().flatten())
                trues.extend(yb.numpy().flatten())
            preds, trues = np.array(preds), np.array(trues)
            rmse = np.sqrt(mean_squared_error(trues, preds))
        print(f'Epoch [{epoch}/{num_epochs}], Loss: {train_losses[-1]:.4f}, RMSE: {rmse:.6f}')

In [ ]:
# Оцінка моделі
model.eval()
with torch.no_grad():
    y_pred = model(X_test_t.to(device)).cpu().numpy().flatten()
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
print('MSE:', mse)
print('MAE:', mae)
print('R2:', r2)

In [ ]:
#  Аналіз результатів
plt.figure(figsize=(6,6))
plt.scatter(y_test, y_pred, alpha=0.6)
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--')
plt.xlabel('Actual Strength')
plt.ylabel('Predicted Strength')
plt.title('Actual vs Predicted')
plt.show()
print()
plt.figure()
plt.plot(train_losses)
plt.xlabel('Epoch')
plt.ylabel('Train Loss (MSE)')
plt.title('Training Loss Curve')
plt.show()

# ***Висновки після експерементальної оптимізації***

## ***Порівняння результатів***


MSE = 257
MAE = 13
R_2 ~ 0.0

------------

*Оптимізована модель (ConcreteNetDeep + Adam + weight_decay=1e-4, 400 епох):*


MSE ≈ 32.9 - відмінний результат (<35)

MAE ≈ 3.81 - відмінний результат (<5)

R² ≈ 0.87 - відмінний результат (>0.8)

------------


***Висновки***

1. Навчання моделі:

 - Оптимізована архітектура з більшою кількістю шарів та нейронів змогла краще захопити нелінійні залежності між компонентами бетону та його міцністю.

 - Використання Adam з weight_decay=1e-4 забезпечило стабільну збіжність і уникнення перенавчання.

 - Збільшення кількості епох до 400 дозволило моделі поступово знизити втрати до оптимального рівня.


2. Результати на валідаційній вибірці:

 - Метрики досягли «відмінних» значень за всіма критеріями (MSE, MAE, R²).

 - Це означає, що модель добре узагальнює і може реально прогнозувати міцність бетону на основі його складу.


3. Оцінка:

 - Глибока архітектура дала значний приріст якості, але може бути більш ресурсомісткою.

 - Можна ще експериментувати з Dropout або BatchNorm для додаткової стабільності.

4. Шляхи покращення:

 - Спробувати різні learning rate (наприклад, 0.0005).

 - Додати регуляризацію (Dropout).

 - Перевірити інші оптимізатори (RMSprop).

 - Виконати крос‑валідацію для більш надійної оцінки.















